# Environment

In [3]:
from optionlib.data import ticker, prices, earnings
from optionlib import *

In [4]:
import os
import pandas as pd
import numpy as np
from scipy.stats import norm
from scipy.optimize import minimize_scalar   
import datetime as dt
from joblib import Parallel, delayed
from plotly import express as px

# Raw data CSV to parquet conversion

## Option chain

In [7]:
%%time
for d in range(2012,2025):
    
    files = [i for i in os.listdir('../CBOE raw data/order_000050203/item_000057965') if str(d) in i]
    
    pd.concat(
        [pd.read_csv(f'../CBOE raw data/order_000050203/item_000057965/{f}',encoding='utf-8') for f in files]
    ).assign(
        quote_datetime = lambda x: pd.to_datetime(x.quote_datetime),
        expiration = lambda x: pd.to_datetime(x.expiration).dt.date
    ).to_parquet(f'../historical_data/spx_option_chain_historical_{d}.parquet')

CPU times: user 8min 13s, sys: 1min 43s, total: 9min 56s
Wall time: 10min 20s


## Price data

In [99]:
prices_raw = pd.concat(
    [pd.read_csv(f'../CBOE raw data/order_000050263/item_000058027/{f}') 
     for f in os.listdir('../CBOE raw data/order_000050263/item_000058027')]
).to_parquet('../historical_data/spx_price_history.parquet')

In [ ]:
tbill_1mo_url_csv = 'https://fred.stlouisfed.org/graph/fredgraph.csv?mode=fred&id=DGS1MO&vintage_date=2024-01-13&revision_date=2024-01-13&nd=1954-07-01'

tbill = pd.read_csv(tbill_1mo_url_csv).rename(
    columns={'DATE':'Date'}
).assign(
    Date = lambda x: pd.to_datetime(x.Date)
).set_index('Date').replace('.',np.nan).astype(float).ffill()

In [ ]:
prices = prices.assign(
    quote_datetime = lambda x: pd.to_datetime(x.quote_datetime),
    Date = lambda x: x.quote_datetime.dt.date
).set_index(['quote_datetime','Date'],drop = False).join(tbill)

In [148]:
prices.to_parquet('../historical_data/spx_price_history.parquet')

# Calculations

## Implied volatility Black Scholes function

In [2]:
N = norm.cdf

def BS_CALL(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * N(d1) - K * np.exp(-r*T)* N(d2)

def BS_PUT(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma* np.sqrt(T)
    return K*np.exp(-r*T)*N(-d2) - S*N(-d1)    
    

def implied_vol(opt_value, S, K, T, r, type_='C'):
    
    def call_obj(sigma):
        return abs(BS_CALL(S, K, T, r, sigma) - opt_value)
    
    def put_obj(sigma):
        return abs(BS_PUT(S, K, T, r, sigma) - opt_value)
    
    if type_ == 'C':
        res = minimize_scalar(call_obj, bounds=(0.01,6), method='bounded')
        return res.x
    elif type_ == 'P':
        res = minimize_scalar(put_obj, bounds=(0.01,6),
                              method='bounded')
        return res.x
    else:
        raise ValueError("type_ must be 'put' or 'call'")

## For a single year

In [5]:
max_T = 30
strike_window = 0.05

oc_2024 = pd.read_parquet('../historical_data/spx_option_chain_historical_2024.parquet')
prices = pd.read_parquet('../historical_data/spx_price_history.parquet')

oc_2024 = oc_2024.assign(
    quote_datetime = lambda x: pd.to_datetime(x.quote_datetime),
    expiration = lambda x: pd.to_datetime(x.expiration) + np.timedelta64(975,'m'),
    time_to_expiry = lambda x: (x.expiration - x.quote_datetime).div(np.timedelta64(1,'D')).astype(float),
    bs_time = lambda x: x.time_to_expiry.div(365)
).set_index(['quote_datetime','expiration','strike','option_type'],drop = False)

oc_2024 = oc_2024.join(prices.rename(columns = {'close':'SPX_open'})[['SPX_open','DGS1MO']])

oc_2024_filtered = oc_2024.loc[
    lambda x: x.open.gt(0)
    & x.bs_time.gt(0)
    & x.time_to_expiry.le(7)
    & x.strike.between(x.SPX_open*(1-strike_window),x.SPX_open*(1+strike_window))
]

In [6]:
%%time
spx_iv = oc_2024_filtered.apply(
    lambda x: implied_vol(
        x.open,
        x.SPX_open,
        x.strike,
        x.bs_time,
        x.DGS1MO,
        x.option_type
    ),axis = 1
)

CPU times: user 1min 26s, sys: 114 ms, total: 1min 27s
Wall time: 1min 27s


## For all years

In [14]:
%%time

max_T = 30
strike_window = 0.05

prices = pd.read_parquet('../historical_data/spx_price_history.parquet')

def process_IV(y):
    oc_temp = pd.read_parquet(f'../historical_data/spx_option_chain_historical_{y}.parquet')

    oc_temp = oc_temp.assign(
        expiration = lambda x: x.expiration + np.timedelta64(975,'m'),
        time_to_expiry = lambda x: (x.expiration - x.quote_datetime).div(np.timedelta64(1,'D')).astype(float),
        bs_time = lambda x: x.time_to_expiry.div(365)
    ).set_index(['quote_datetime','expiration','strike','option_type'],drop = False)

    oc_temp = oc_temp.join(prices.rename(columns = {'close':'SPX_open'})[['SPX_open','DGS1MO']])

    oc_temp = oc_temp.loc[
        lambda x: x.open.gt(0)
        & x.bs_time.gt(0)
        & x.time_to_expiry.le(max_T)
        & x.strike.between(x.SPX_open*(1-strike_window),x.SPX_open*(1+strike_window))
    ]
    
    oc_out = oc_temp.apply(
        lambda x: implied_vol(x.open,x.SPX_open,x.strike,x.bs_time,0.05,x.option_type),
        axis = 1
    )
    
    oc_out.rename('IV').to_frame().to_parquet(f'../historical_data/spx_iv_{y}_backup_30d.parquet')
    
    return oc_out

IV_dict = Parallel(n_jobs=4,verbose = 5)(delayed(process_IV)(i) for i in range(2012,2025))

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
/Users/dwynne891/opt/anaconda3/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/Users/dwynne891/opt/anaconda3/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/Users/dwynne891/opt/anaconda3/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
[Parallel(n_jobs=4)]: Done   9 out of  13 | elapsed: 86.3min remaining: 38.3min


CPU times: user 517 ms, sys: 588 ms, total: 1.11 s
Wall time: 2h 44min 23s


[Parallel(n_jobs=4)]: Done  13 out of  13 | elapsed: 164.4min finished


In [ ]:
iv = pd.concat(
    [pd.read_parquet(f'../historical_data/{i}') for i in os.listdir(f'../historical_data/') if 'backup_30d.parquet' in i]
)

iv = iv.groupby(iv.index.names).mean()

iv.to_parquet('../historical_data/spx_iv_all_30d.parquet')

## Save IV series

In [175]:
iv = pd.read_parquet('../historical_data/spx_iv_all_30d.parquet')

def IV_transform(y):
    print(f'Loading {y} data...')
    prices = pd.read_parquet('../historical_data/spx_price_history.parquet')
    oc_temp = pd.read_parquet(
        f'../historical_data/spx_option_chain_historical_{y}.parquet',
        columns = ['quote_datetime','expiration','option_type','strike','open'],
        filters=[
            ('root','==','SPXW'),
            ('strike','>',prices.where(lambda x: x.quote_datetime.dt.year==y).open.min()*.95),
            ('strike','<',prices.where(lambda x: x.quote_datetime.dt.year==y).open.max()*1.05)
        ]
    )
    print(f'Transforming {y} data...')
    
    oc_temp = oc_temp.assign(
        expiration = lambda x: pd.to_datetime(x.expiration) + np.timedelta64(975,'m'),
        days_to_expiry = lambda x: np.floor(
            (x.expiration.dt.date - x.quote_datetime.dt.date).div(np.timedelta64(1,'D'))
        ).astype(int)
    ).set_index(
        ['quote_datetime','expiration','option_type']
    ).set_index('strike',append = True,drop = False)
    
    oc_temp = oc_temp.join(prices.rename(columns = {'close':'SPX_open'})[['SPX_open','DGS1MO']])
    
    oc_temp = oc_temp.assign(
        strike_distance = lambda x: abs(x.strike - x.SPX_open)
    )
    
    strike_dist_min_idx = oc_temp.groupby(['quote_datetime','expiration','option_type']).strike_distance.idxmin()
    
    oc_temp = oc_temp.loc[strike_dist_min_idx]
    
    oc_temp = oc_temp.join(iv,how='inner')

    print(f'{y} complete')
    return oc_temp
    
IV_dict = Parallel(
    n_jobs=4,
    verbose = 5
)(delayed(IV_transform)(i) for i in range(2012,2025))

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


Loading 2012 data...Loading 2013 data...

Loading 2014 data...Loading 2015 data...

Transforming 2012 data...
Transforming 2013 data...
Transforming 2014 data...
Transforming 2015 data...
Loading 2016 data...
Transforming 2016 data...
Loading 2017 data...
Transforming 2017 data...
Loading 2018 data...
Transforming 2018 data...
Loading 2019 data...
Transforming 2019 data...
Loading 2020 data...
Transforming 2020 data...


/Users/dwynne891/opt/anaconda3/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Loading 2021 data...
Transforming 2021 data...


/Users/dwynne891/opt/anaconda3/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Loading 2022 data...
Transforming 2022 data...
Loading 2023 data...
Transforming 2023 data...


[Parallel(n_jobs=4)]: Done   9 out of  13 | elapsed:  2.3min remaining:  1.0min


Loading 2024 data...
Transforming 2024 data...


[Parallel(n_jobs=4)]: Done  13 out of  13 | elapsed:  3.2min finished


In [179]:
iv_timeseries = pd.concat(IV_dict).droplevel(
    ['expiration','strike','Date']
).set_index('days_to_expiry',append = True)[['IV']].unstack('days_to_expiry')

In [180]:
iv_timeseries.to_parquet('../historical_data/iv_timeseries.parquet')

In [210]:
iv_timeseries.iloc[1,:].dropna().values

array([0.24189978])

In [230]:
collapse_dict = dict()
for i in iv_timeseries.index:
    values = iv_timeseries.loc[i,:].dropna().values
    values = np.append(values,[np.nan]*(30-len(values)))
    collapse_dict[i] = values

In [237]:
iv_timeseries

IV                              \
days_to_expiry                         0   1   2         3         4    
quote_datetime      option_type                                         
2012-01-03 10:30:00 C                 NaN NaN NaN  0.162501       NaN   
                    P                 NaN NaN NaN  0.241900       NaN   
2012-01-03 11:30:00 C                 NaN NaN NaN  0.198116       NaN   
                    P                 NaN NaN NaN  0.211975       NaN   
2012-01-03 12:30:00 C                 NaN NaN NaN  0.218104       NaN   
...                                   ...  ..  ..       ...       ...   
2024-01-12 14:30:00 P            0.272338 NaN NaN       NaN  0.081697   
2024-01-12 15:30:00 C            0.495551 NaN NaN       NaN  0.078661   
                    P            0.083288 NaN NaN       NaN  0.053583   
2024-01-12 16:15:00 C                 NaN NaN NaN       NaN  0.043830   
                    P                 NaN NaN NaN       NaN  0.080371   

                                                                       ...  \
days_to_expiry                         5         6         7   8   9   ...   
quote_datetime      option_type                                        ...   
2012-01-03 10:30:00 C                 NaN       NaN       NaN NaN NaN  ...   
                    P                 NaN       NaN       NaN NaN NaN  ...   
2012-01-03 11:30:00 C                 NaN       NaN       NaN NaN NaN  ...   
                    P                 NaN       NaN       NaN NaN NaN  ...   
2012-01-03 12:30:00 C                 NaN       NaN       NaN NaN NaN  ...   
...                                   ...       ...       ...  ..  ..  ...   
2024-01-12 14:30:00 P            0.094820  0.091994  0.092068 NaN NaN  ...   
2024-01-12 15:30:00 C            0.085461  0.088451  0.096047 NaN NaN  ...   
                    P            0.067945  0.073362  0.076520 NaN NaN  ...   
2024-01-12 16:15:00 C            0.061174  0.074346  0.077200 NaN NaN  ...   
                    P            0.086129  0.095475  0.092981 NaN NaN  ...   

                                                                         \
days_to_expiry                         21  22  23  24        25  26  27   
quote_datetime      option_type                                           
2012-01-03 10:30:00 C                 NaN NaN NaN NaN       NaN NaN NaN   
                    P                 NaN NaN NaN NaN       NaN NaN NaN   
2012-01-03 11:30:00 C                 NaN NaN NaN NaN       NaN NaN NaN   
                    P                 NaN NaN NaN NaN       NaN NaN NaN   
2012-01-03 12:30:00 C                 NaN NaN NaN NaN       NaN NaN NaN   
...                                   ...  ..  ..  ..       ...  ..  ..   
2024-01-12 14:30:00 P            0.107764 NaN NaN NaN  0.104286 NaN NaN   
2024-01-12 15:30:00 C            0.110545 NaN NaN NaN  0.101651 NaN NaN   
                    P            0.107273 NaN NaN NaN  0.109288 NaN NaN   
2024-01-12 16:15:00 C            0.100641 NaN NaN NaN       NaN NaN NaN   
                    P            0.111209 NaN NaN NaN       NaN NaN NaN   

                                                   
days_to_expiry                         28  29  30  
quote_datetime      option_type                    
2012-01-03 10:30:00 C                 NaN NaN NaN  
                    P                 NaN NaN NaN  
2012-01-03 11:30:00 C                 NaN NaN NaN  
                    P                 NaN NaN NaN  
2012-01-03 12:30:00 C                 NaN NaN NaN  
...                                   ...  ..  ..  
2024-01-12 14:30:00 P                 NaN NaN NaN  
2024-01-12 15:30:00 C            0.108888 NaN NaN  
                    P            0.106463 NaN NaN  
2024-01-12 16:15:00 C            0.099659 NaN NaN  
                    P            0.109854 NaN NaN  

[41805 rows x 31 columns]

In [235]:
iv_timeseries_trading_days = pd.DataFrame.from_dict(collapse_dict,orient = 'index')

In [239]:
iv_timeseries_trading_days.index = pd.MultiIndex.from_tuples(
    iv_timeseries_trading_days.index,
    names=['quote_datetime','option_type']
)

In [251]:
iv_timeseries_trading_days = iv_timeseries_trading_days.rename(columns = lambda x: f'IV_{x}')

In [254]:
iv_timeseries_trading_days.to_parquet('../historical_data/iv_timeseries_trading_days.parquet')